In [5]:
!pip install --quiet geopy

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cvxpy as cp
from geopy import distance

C = [
        [0, 0.2, 0.4, 0.6, 0.3, 0.2, 0.5, 0.4, 0.6, 0.3],
        [0.2, 0, 0.2, 0.4, 0.2, 0.3, 0.4, 0.5, 0.5, 0.5],
        [0.4, 0.2, 0, 0.2, 0.2, 0.4, 0.3, 0.5, 0.5, 0.6],
        [0.6, 0.4, 0.2, 0, 0.3, 0.4, 0.2, 0.4, 0.3, 0.6],
        [0.3, 0.2, 0.2, 0.3, 0, 0.2, 0.2, 0.3, 0.4, 0.4],
        [0.2, 0.3, 0.4, 0.4, 0.2, 0, 0.3, 0.2, 0.5, 0.2],
        [0.5, 0.4, 0.3, 0.2, 0.2, 0.3, 0, 0.2, 0.2, 0.5],
        [0.4, 0.5, 0.5, 0.4, 0.3, 0.2, 0.2, 0, 0.3, 0.3],
        [0.6, 0.5, 0.5, 0.3, 0.4, 0.2, 0.2, 0.3, 0, 0.6],
        [0.3, 0.5, 0.6, 0.6, 0.4, 0.5, 0.5, 0.3, 0.6, 0]
    ]

n=10
shape = np.shape(C)
X = cp.Variable(shape, boolean=True)
u = cp.Variable(n, nonneg=True)
ones = np.ones((n,1))

# Defining the objective function
objective = cp.Minimize(cp.sum(cp.multiply(C, X)))

# Defining the constraints
constraints = []
constraints += [X @ ones == ones]
constraints += [X.T @ ones == ones]
constraints += [cp.diag(X) == 0]
constraints += [u[1:] >= 2]
constraints += [u[1:] <= n]
constraints += [u[0] == 1]

for i in range(1, n):
    for j in range(1, n):
        if i != j:
            constraints += [ u[i] - u[j] + 1  <= (n - 1) * (1 - X[i, j]) ]

# Solving the problem
prob = cp.Problem(objective, constraints)
prob.solve(solver=cp.GUROBI,verbose = False)

# Transforming the solution to a path
X_sol = np.argwhere(X.value==1)
order = X_sol[0].tolist()

for i in range(1, n):
    row = order[-1]
    order.append(X_sol[row,1])

# Showing the optimal path
print('The path is:\n')
print( ' => '.join(map(str, order)))

distance = np.sum(np.multiply(C, X.value))
print('The optimal distance is:', np.round(distance,2), 'mile')

The path is:

0 => 1 => 4 => 2 => 3 => 8 => 6 => 7 => 9 => 5 => 0
The optimal distance is: 2.5 mile


In [3]:
import numpy as np
from python_tsp.exact import solve_tsp_dynamic_programming

distance = np.array([
        [0, 0.2, 0.4, 0.6, 0.3, 0.2, 0.5, 0.4, 0.6, 0.3],
        [0.2, 0, 0.2, 0.4, 0.2, 0.3, 0.4, 0.5, 0.5, 0.5],
        [0.4, 0.2, 0, 0.2, 0.2, 0.4, 0.3, 0.5, 0.5, 0.6],
        [0.6, 0.4, 0.2, 0, 0.3, 0.4, 0.2, 0.4, 0.3, 0.6],
        [0.3, 0.2, 0.2, 0.3, 0, 0.2, 0.2, 0.3, 0.4, 0.4],
        [0.2, 0.3, 0.4, 0.4, 0.2, 0, 0.3, 0.2, 0.5, 0.2],
        [0.5, 0.4, 0.3, 0.2, 0.2, 0.3, 0, 0.2, 0.2, 0.5],
        [0.4, 0.5, 0.5, 0.4, 0.3, 0.2, 0.2, 0, 0.3, 0.3],
        [0.6, 0.5, 0.5, 0.3, 0.4, 0.2, 0.2, 0.3, 0, 0.6],
        [0.3, 0.5, 0.6, 0.6, 0.4, 0.5, 0.5, 0.3, 0.6, 0]
    ])

permutation, distance = solve_tsp_dynamic_programming(distance)
print("optimal path: ", permutation)
print("total distance: ", distance, "mile")

optimal path:  [0, 1, 4, 2, 3, 6, 8, 5, 7, 9]
total distance:  2.1999999999999997 mile


Since the tsp(travel salesman package) prints out permutation that doesn't go back to the starting point at the end, I will add the path from node 9 to node 0(starting point) to the total distance to check if the result of tsp matches with the result using Gurobi.\
By adding the path between node 9 and node 0 to the total distance, the total distance will be 2.4999999999999997 mile. The optimal value does not equal to 2.5, but very close to 2.5. This is because there can be an error when doing calculations of float values in python.\
You can see that the paths from two outputs are slightly different. There can be more than one possible optimal path, so the result(path) between using gurobi and tsp package can differ. Even though the paths are different, the optimal value seems to equal as 2.5 mile.